In [2]:
import cv2
import pickle
import os
import numpy as np
import mysql.connector

# Function to connect to the MySQL database
def connect_db():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='Ashu@1996',
        database='myproject'
    )

# Function to insert eye data into the database
def insert_eye_data(adhaar_number, eye):
    conn = connect_db()
    cursor = conn.cursor()
    
    # SQL query to insert data into the voterdata table
    add_eye = ("INSERT INTO eyedata "
               "(adhar_number, eye) "
               "VALUES (%s, %s)")
    
    data_eye = (adhaar_number, eye)
    cursor.execute(add_eye, data_eye)
    
    conn.commit()
    cursor.close()
    conn.close()

# Initialize video capture from the default camera (0)
video = cv2.VideoCapture(0)
# Load the Haar Cascade classifier for eye detection
eye_cascade = cv2.CascadeClassifier("c:/users/user/anaconda3/lib/site-packages/cv2/data/haarcascade_eye.xml")

eyes_data = []
i = 0
name = input('Enter your ADHAAR Number: ')
framesTotal = 21  # Total number of frames to capture
captureAfterFrame = 4  # Capture an image every 'captureAfterFrame' frames

while True:
    ret, frame = video.read()  # Capture frame-by-frame from the video feed
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert the frame to grayscale for eye detection
    eyes = eye_cascade.detectMultiScale(gray, 1.3, 5)  # Detect eyes in the grayscale image
    
    # Iterate over detected eyes
    for (x, y, w, h) in eyes:
        crop_img = frame[y:y+h, x:x+w]  # Crop the detected eye from the frame
        resized_img = cv2.resize(crop_img, (50, 50))  # Resize the eye image to 50x50 pixels
        
        # Capture image every 'captureAfterFrame' frames until the total frames are reached
        if len(eyes_data) <= framesTotal and i % captureAfterFrame == 0:
            eyes_data.append(resized_img)  # Add the resized image to the list
        
        i += 1
        # Display the number of captured eye images on the frame
        cv2.putText(frame, str(len(eyes_data)), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (50, 50, 255), 1)
        # Draw a rectangle around the detected eye
        cv2.rectangle(frame, (x, y), (x+w, y+h), (50, 50, 255), 1)
    
    cv2.imshow('frame', frame)
    
    k = cv2.waitKey(1)
    # Exit loop if 'q' is pressed or enough frames are captured
    if k == ord('q') or len(eyes_data) >= framesTotal:
        break

video.release()  # Release the video capture object
cv2.destroyAllWindows()  # Close all OpenCV windows

print(f"Captured {len(eyes_data)} eye images")  # Print the number of captured eye images

# Convert each eye image to bytes and store it in the database
for img in eyes_data:
    success, img_encoded = cv2.imencode('.jpg', img)  # Encode the image as JPEG
    if success:
        img_bytes = img_encoded.tobytes()  #Convert encoded image to bytes
        insert_eye_data(name, img_bytes)  #Insert image data into the database

print("Eye data has been stored in the database.")  #Confirmation message


Enter your ADHAAR Number:  12345


Captured 21 eye images
Eye data has been stored in the database.
